In [6]:
from diffusion import SequentialDiffusionEquation, OMPdiffusionEquation, CUDADiffusionEquation
import math
import numpy as np

### Simulation Parameters

In [7]:
# Parameters
library = "../build/libDiffusionEquation.so"
N = 2000
D = 0.1
dt = 0.01
dx = 1.0

In [3]:
solver = CUDADiffusionEquation(
    library_path="../libcud.diffusion.so",
    N=2000,
    D=0.1,
    DELTA_T=0.01,
    DELTA_X=1.0,
)

# Perform the computation
for t in range(500):
    avg_diff = solver.step()
    if t % 100 == 0:
        print(f"Step {t}: Average Difference = {avg_diff}")

# Retrieve the result
solver.get_result()
final_concentration = solver.concentration_matrix

# Finalize to free device memory
solver.finalize()

Step 0: Average Difference = 2.00400610319519e-09
Step 100: Average Difference = 1.2324806778815408e-09
Step 200: Average Difference = 7.817943429761211e-10
Step 300: Average Difference = 5.115284590805102e-10
Step 400: Average Difference = 4.2163164115354466e-10


In [8]:
def create_models():
    # Initial concentration
    center = N // 2
    initial_concentration = {(center, center): 1.0}

    # Create the Sequential Diffusion Equation
    sequential = SequentialDiffusionEquation(
        library,
        initial_concentration_points=initial_concentration,
        N=N,
        D=D,
        DELTA_T=dt,
        DELTA_X=dx,
    )

    # Create the Parallel Diffusion Equation
    omp = OMPdiffusionEquation(
        library,
        initial_concentration_points=initial_concentration,
        N=N,
        D=D,
        DELTA_T=dt,
        DELTA_X=dx,
    )
    
    cuda = CUDADiffusionEquation(
        library_path="../libcud.diffusion.so",
        N=N,
        D=D,
        DELTA_T=dt,
        DELTA_X=dx,
    )

    return sequential, omp, cuda

### Verify if the results are the same as the expected results in project description

In [9]:
sequential, omp, cuda = create_models()
for i in range(500):
    seq_diff = sequential.step()
    omp_diff = omp.step()
    cuda_diff = cuda.step()

    if i % 100 == 0:
        print(f"{i}: Sequential: {seq_diff} -- OMP: {omp_diff} -- CUDA: {cuda_diff}")

cuda.get_result()
conc_center_seq = sequential.concentration_matrix[N // 2, N // 2]
conc_center_omp = omp.concentration_matrix[N // 2, N // 2]
conc_center_cuda = cuda.concentration_matrix[N // 2, N // 2]
print(f"Concentration at center: {conc_center_seq} -- {conc_center_omp} -- {conc_center_cuda}")

cuda.finalize()

0: Sequential: 2.004006008010013e-09 -- OMP: 2.004006008010013e-09 -- CUDA: 2.00400610319519e-09
100: Sequential: 1.2324806785661765e-09 -- OMP: 1.2324806785661763e-09 -- CUDA: 1.2324806778815408e-09
200: Sequential: 7.817943347345305e-10 -- OMP: 7.817943347345306e-10 -- CUDA: 7.817943429761211e-10
300: Sequential: 5.115284509398786e-10 -- OMP: 5.115284509398781e-10 -- CUDA: 5.115284590805102e-10
400: Sequential: 4.216316312010041e-10 -- OMP: 4.2163163120100405e-10 -- CUDA: 4.2163164115354466e-10
Concentration at center: 0.21651223169500305 -- 0.21651223169500305 -- 0.21651223169500305


### Verify if the Sequential and OMP solutions are the same

In [6]:
error = False
sequential, parallel = create_models()
for i in range(500):
    seq_diff = sequential.step()
    omp_diff = parallel.step()

    if i % 100 == 0:
        print(f"Iteration {i}: OMP: {omp_diff}, Sequential: {seq_diff}")

    if not math.isclose(seq_diff, omp_diff):
        print(f"Error in iteration {i}: OMP and Sequential results are different")
        print(f"OMP: {omp_diff}, Sequential: {seq_diff}")
        error = True

    if np.any(parallel.concentration_matrix != sequential.concentration_matrix):
        print(f"Error in iteration {i}: OMP and Sequential matrix are different")
        error = True

print("All iterations completed successfully")
if not error:
    print("No errors found")

ValueError: too many values to unpack (expected 2)

In [ ]:
def carregar_matriz(nome_arquivo):
    try:
        return np.loadtxt(nome_arquivo)
    except Exception as e:
        print(f"Erro ao carregar o arquivo {nome_arquivo}: {e}")
        return None


def comparar_matrizes(matriz1, matriz2):
    if matriz1.shape != matriz2.shape:
        print("As matrizes têm tamanhos diferentes e não podem ser comparadas.")
        return

    iguais = np.sum(matriz1 == matriz2)
    total = matriz1.size
    porcentagem_iguais = (iguais / total) * 100

    print(f"Porcentagem de valores iguais: {porcentagem_iguais:.2f}%")
    print(
        f"Total de elementos: {total}, Iguais: {iguais}, Diferentes: {total - iguais}"
    )


if __name__ == "__main__":
    matriz_sequential = carregar_matriz("matriz_sequential.txt")
    matriz_omp = carregar_matriz("matriz_omp.txt")

    if matriz_sequential is not None and matriz_omp is not None:
        comparar_matrizes(matriz_sequential, matriz_omp)